<h1>Microsoft Stock Analysis</h1>
The objective of this article is to provide some insights on the performance of Microsoft's stock.


<h3>Library imports</h3>
We start by importing some libraries that will be used.

In [17]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# For reading stock data from yahoo
from pandas_datareader import DataReader

# For time stamps
from datetime import datetime

In order to easily plot pandas dataframes we use cufflinks:
https://github.com/santosjorge/cufflinks

In order to display better quality graphs we use plotly:
https://plotly.com/python/

Run the cell below if you want to have the interactive plots. Keep in mind that you have to install the libraries at your command line/terminal using:

pip install plotly <br>
pip install cufflinks

NOTE: Make sure you only have one installation of Python on your computer when you do this, otherwise the installation may not work.

In [18]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
print(__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

4.6.0


Specify which stock data to get

In [27]:
stocks = ['MSFT']

# Specify start and end times
# Satya Nadella was announced as CEO on the 4th of February 2014
start = datetime(2014,2,4)
end = datetime.now()


#For loop for grabing yahoo finance data and setting as a dataframe
for stock in stocks:   
    # Set DataFrame as the Stock Ticker
    # This creates a dataframe named after the corresponding stock
    globals()[stock] = DataReader(stock,'yahoo',start,end)

Example of a collected dataframe:

In [21]:
MSFT.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2006-01-03,27.000000,26.10,26.250000,26.840000,79973000.0,19.657124
2006-01-04,27.080000,26.77,26.770000,26.969999,57975600.0,19.752342
2006-01-05,27.129999,26.91,26.959999,26.990000,48245500.0,19.766989
2006-01-06,27.000000,26.49,26.889999,26.910000,100963000.0,19.708401
2006-01-09,27.070000,26.76,26.930000,26.860001,55625000.0,19.671768


In [22]:
MSFT.describe()

,High,Low,Open,Close,Volume,Adj Close
count,3607.000000,3607.000000,3607.000000,3607.000000,3.607000e+03,3607.000000
mean,51.031131,50.062984,50.555043,50.568328,4.825143e+07,45.966395
std,36.417107,35.666789,36.067600,36.069177,3.010192e+07,37.612102
min,15.620000,14.870000,15.200000,15.150000,7.425600e+06,11.699240
25%,27.634999,27.145000,27.340000,27.355000,2.776315e+07,21.355717
50%,32.799999,32.150002,32.490002,32.590000,4.251190e+07,26.523193
75%,58.075001,57.434999,57.875000,57.889999,5.998210e+07,54.203138
max,190.699997,186.470001,190.649994,188.699997,5.910522e+08,188.185989


In [23]:
MSFT.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3607 entries, 2006-01-03 to 2020-05-01
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       3607 non-null   float64
 1   Low        3607 non-null   float64
 2   Open       3607 non-null   float64
 3   Close      3607 non-null   float64
 4   Volume     3607 non-null   float64
 5   Adj Close  3607 non-null   float64
dtypes: float64(6)
memory usage: 197.3 KB


<h2>Microsoft stock plot</h2>

In [55]:
# Using the close price
ms_graph = MSFT.loc[:,'Close'].iplot(title='Closing Price')

We can also use Adjusted closing prices of stocks. Adjusted Closing Prices accurately reflect the stocks' value after accounting for any corporate actions such as stock splits, dividends / distributions and rights offerings.

In [56]:
ms_adj_close = ms_graph = MSFT.loc[:,'Adj Close'].iplot(title='Adjusted Closing Price')